In [1]:
params_notebook_name = "datasets.eclip.py.ipynb"
params_resource_dir = "../resources/"
params_subset_chroms = [
    "chr12",
]

params_subset_rbp_cl = [
    "QKI_HepG2",
    "QKI_K562",
    "PTBP1_HepG2",
    "SRSF7_HepG2",
]

params_tmp_dir = None

# Datasets - eCLIP datasets

## Overview

## Imports

In [2]:
import dataclasses
import os
import sys
import tempfile
from io import StringIO
from numbers import Number
from pathlib import Path
from typing import Literal

import numpy as np
import pandas as pd
import pybedtools as pbt
import pyBigWig as pbw
import pydantic.dataclasses
import pyfaidx
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from dotenv import find_dotenv, load_dotenv
from IPython.display import HTML, SVG, Image, display
from pydantic.dataclasses import dataclass
from snakemake.io import glob_wildcards

In [3]:
import matplotlib as mpl
import matplotlib.font_manager
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import rc

try:
    matplotlib.font_manager.findfont("Arial")
    mpl.rcParams["font.sans-serif"] = ["Arial"]
except Exception as e:
    print("Setting Arial font failed in some way...")
    print(e)

rc("text", usetex=False)

In [4]:
def assert_notebook_working_dir(expected_local_file: os.PathLike) -> Path:
    """Assert or try updating the current working directory to where the notebook is located, to enable relative paths references.

    This function is used in a set-up where notebooks are contained within a project
    directory structure in which we want to reference filepaths relative to the notebook.
    e.g. "../src" or "../resources" should be accessible if the notebook is in
    "../notebooks/<notebook_name>.ipynb".

    The function first check the filepath of the expected local file relative
    to the current working directory.

    If not found, the function will try to use the VSCode Jupyter variable `__vsc_ipynb_file__`
    which should report the path of the notebook file being executed.

    It then checks if the expected local file exists, relative to the new working directory.


    Args:
        expected_local_file (os.PathLike): The expected local file to check for in the current working directory.
            This can be the name of the notebook file.

    Raises:
        KeyError: if the `__vsc_ipynb_file__` variable is not found in the global scope, while the first CWD check failed.
        FileNotFoundError: if the expected local file is not found in the current working directory after attempting to change it.
    """
    import os
    from pathlib import Path

    cwd = Path(os.getcwd())
    expected_local_filepath = cwd / expected_local_file

    if not expected_local_filepath.exists():
        if "__vsc_ipynb_file__" not in globals():
            raise KeyError(
                f"Detected CWD: {cwd} ; CWD does not contain expected file, but cannot use __vsc_ipynb_file__ to recover."
            )
        else:
            os.chdir(Path(globals()["__vsc_ipynb_file__"]).parent)
            cwd = Path(os.getcwd())
            print(f"Changed CWD to {cwd}")

            expected_local_filepath = cwd / expected_local_file
            if not expected_local_filepath.exists():
                raise FileNotFoundError(
                    f"Updated (using __vsc_ipynb_file__) CWD: {cwd} ; CWD does not contain expected file."
                )

            return cwd
    else:
        print(f"Confirmed CWD: {cwd} contains expected file: {expected_local_file}")
        return cwd


expected_local_file: str = params_notebook_name
cwd = assert_notebook_working_dir(expected_local_file=expected_local_file)
print(cwd)

Confirmed CWD: /home/l10n/projects/ml4rg25-parnet/parnet_demo/notebooks contains expected file: datasets.eclip.py.ipynb
/home/l10n/projects/ml4rg25-parnet/parnet_demo/notebooks


## Initialization

In [5]:
# Env variables, defined in the `.env` file at the root of the project directory.
load_dotenv(find_dotenv(), override=True)

False

In [7]:
resource_dir = Path(params_resource_dir)
if not resource_dir.exists():
    raise FileNotFoundError("Resource directory does not exist: " + str(resource_dir))

print("Using resources from:", resource_dir)

Using resources from: ../resources


In [8]:
bed6_cols = ["chrom", "start", "end", "name", "score", "strand"]

In [9]:
# Caution: pyBedTools relies on tmp dir, which can get quite large.
def create_randomized_tmp_dir() -> str:
    # Get the parent tmp dir where to create a randomized tmp dir.

    parent_tmp_dir = None

    # Default system value
    parent_tmp_dir = Path(tempfile.gettempdir())
    if params_tmp_dir is not None:
        parent_tmp_dir = Path(params_tmp_dir)

    if os.getenv("TMP_DIR") is not None:
        parent_tmp_dir = Path(os.getenv("TMP_DIR"))

    if os.getenv("TMP") is not None:
        parent_tmp_dir = Path(os.getenv("TMP"))

    if parent_tmp_dir is None:
        raise ValueError("No temporary directory specified or found.")

    # Test write access to the parent tmp dir
    if not parent_tmp_dir.is_dir():
        raise NotADirectoryError(f"Parent tmp dir is not a directory: {parent_tmp_dir}")

    if not os.access(parent_tmp_dir, os.W_OK):
        raise PermissionError(f"Parent tmp dir is not writable: {parent_tmp_dir}")

    # Make a randomized tmp dir
    tmp_dir = tempfile.mkdtemp(dir=parent_tmp_dir)
    return tmp_dir


tmp_dir = create_randomized_tmp_dir()
print("Using temporary directory for pyBedTools:", tmp_dir)

pbt.set_tempdir(tmp_dir)

Using temporary directory for pyBedTools: /tmp/tmp5hbkvhqa


## Load general datasets

### Genome data

In [10]:
filepath = resource_dir / "general" / "genome" / "hg38.chrom.sizes"
print('Loading the "genome file" from:', filepath)

genome_file = pd.read_csv(filepath, header=None, sep="\t")
genome_file.columns = ["chrom", "size"]

main_autosomes = [f"chr{i}" for i in range(1, 23)]
sex_chromosomes = ["chrX", "chrY"]
mitochondrial_chromosome = ["chrM"]
main_chromosomes = list(set(main_autosomes) | set(sex_chromosomes) | set(mitochondrial_chromosome))


Loading the "genome file" from: ../resources/general/genome/hg38.chrom.sizes


In [11]:
# We also load the genome FASTA file, to demonstrate basic sequence enrichment analysis.
fa = pyfaidx.Fasta(resource_dir / "general" / "genome" / "hg38.fa.gz")

### Gene annotations

We will load MANE reference transcripts and genome chrom sizes to define ranges of regions to look into.

In [12]:
filepath = (
    resource_dir / "general" / "gene_annotations.hg38.MANE_v1.4" / "MANE.GRCh38.v1.4.ensembl_genomic.transcripts.bed.gz"
)

# Rather than load the file, we will query it with bedtools,
bt_transcripts = pbt.BedTool(filepath)
bt_transcripts = bt_transcripts.tabix()

bt_genome = pbt.BedTool.from_dataframe(
    genome_file.loc[lambda df: df["chrom"].isin(params_subset_chroms), :]
    .assign(start=0, end=lambda df: df["size"])
    .loc[:, ["chrom", "start", "end"]]
)


mane_transcripts = bt_transcripts.intersect(bt_genome).to_dataframe()

In [13]:
display(mane_transcripts.head(3))
print("Number of MANE transcripts:", len(mane_transcripts))

,chrom,start,end,name,score,strand
0,chr12,66766,178455,ENST00000538872.6,.,+
1,chr12,190080,214157,ENST00000684302.1,.,-
2,chr12,220621,262836,ENST00000343164.9,.,-


Number of MANE transcripts: 1008


We also load the GENCODE datasets:

- gene metadata table
- gene type metadata table
- non-overlapping, biotype annotated intervals for plus and minus strands

In [14]:
filepath = resource_dir / "general" / "gene_annotations.hg38.gencode_v40" / "gene_metadata.tsv.gz"
gencode_gene_metadata = pd.read_csv(filepath, sep="\t", compression="gzip", header=0)
display(gencode_gene_metadata.head(3))

,transcript_id,gene_id,gene_name,gene_type,protein_id,official_name,official_id
0,ENST00000456328.2,ENSG00000223972.5,DDX11L1,transcribed_unprocessed_pseudogene,NaN,DDX11L1,HGNC:37102
1,ENST00000450305.2,ENSG00000223972.5,DDX11L1,transcribed_unprocessed_pseudogene,NaN,DDX11L1,HGNC:37102
2,ENST00000488147.1,ENSG00000227232.5,WASH7P,unprocessed_pseudogene,NaN,WASH7P,HGNC:38034


In [15]:
filepath = resource_dir / "general" / "gene_annotations.hg38.gencode_v40" / "simplified_grouped_genetype.tsv"
gencode_genetype_metadata = pd.read_csv(filepath, sep="\t", header=0)
display(gencode_genetype_metadata.head(3))


,genetype,simplified_genetype,group_genetype
0,IG_C_gene,IG_TR_gene,protein_coding
1,IG_D_gene,IG_TR_gene,protein_coding
2,IG_J_gene,IG_TR_gene,protein_coding


In [16]:
filepath_minus = (
    resource_dir / "general" / "gene_annotations.hg38.gencode_v40" / "complete.non-overlap.annotated.minus.bed.gz"
)
filepath_plus = (
    resource_dir / "general" / "gene_annotations.hg38.gencode_v40" / "complete.non-overlap.annotated.plus.bed.gz"
)

# Rather than load the complete files, we apply again bedtools to query the files.
gencode_minus = pbt.BedTool(filepath_minus).intersect(bt_genome).to_dataframe()
gencode_plus = pbt.BedTool(filepath_plus).intersect(bt_genome).to_dataframe()

display(gencode_minus.head(3))
display(gencode_plus.head(3))
print(f"Number of GENCODE minus strand intervals: {len(gencode_minus):_}")
print(f"Number of GENCODE plus strand intervals: {len(gencode_plus):_}")

,chrom,start,end,name,score,strand
0,chr12,36601,38133,lncRNA;FAM138D;lncRNA;ENSG00000249054.2,.,-
1,chr12,106523,111850,lncRNA;ENSG00000256948;lncRNA;ENSG00000256948.1,.,-
2,chr12,137410,149169,lncRNA;ENSG00000249695;lncRNA;ENSG00000249695.6,.,-


,chrom,start,end,name,score,strand
0,chr12,66766,66882,five_prime_UTR;IQSEC3;protein_coding;ENSG00000...,.,+
1,chr12,66882,66885,start_codon;IQSEC3;protein_coding;ENSG00000120...,.,+
2,chr12,66885,67436,CDS;IQSEC3;protein_coding;ENSG00000120645.12,.,+


Number of GENCODE minus strand intervals: 26_540
Number of GENCODE plus strand intervals: 26_688


## eCLIP data - read counts

Here we load `bigwig` files, a binary format for storing one numeric value per position
in a genomic coordinate system.

This format can be manually built from a `Wiggle` file format, using the command `wigToBigWig <in.wig> <genome.chrom.sizes> <out.bw>`.

Check the specification of the `Wiggle` format [here](https://genome.ucsc.edu/goldenPath/help/wiggle)

The simplest version below shows a header line defining a fixed-step size of 1 nucleotide, starting at position 1 (wiggle format is 1-based) or chromosome 1.

```bash
fixedStep  chrom="chr1" start=1  step=1`
0.1
0.2
...
```

This can be useful for creating your own `bigwig` files, for example from PARNET predictions.

Hereafter we demonstrate how to manipulate `bigwig` files using the `pyBigWig` library.

On the available files:

- each eCLIP experiment is performed on a specific RNA Binding Protein (e.g. `QKI`) in a specific Cell Line (e.g. `HepG2`).

- each experiment has
    - 2 biological replicates for the *eCLIP* (i.e. the data generated from the part of the protocol where the RBP is actually immunoprecipitated),
    - and 1 biological replicate for the *input* (i.e. the data generated from the part of the protocol where no RBP is immunoprecipitated).

- for training PARNET, the two biological replicates of the *eCLIP* are summed position-wise into a single file. Keeping the two replicate separate here allows to compare between-replicate correlation vs prediction-vs-replicate correlation.

- For now, we will simply demonstrate how to load the data.

In [17]:
LIST_ECLIP_STRAND: list[Literal["pos", "neg"]]
LIST_ECLIP_REPLICATE: list[Literal["1", "2"]]
LIST_SUBSET_RBP_CL: list[str] = params_subset_rbp_cl

ufmt_readcounts_eclip_fp = (
    resource_dir
    / "parnet_encore_eclip"
    / "bigwig_processed_counts"
    / "{RBP_CL}"
    / "eCLIP"
    / "{REPLICATE}"
    / "counts.{STRAND}.bw"
)

ufmt_readcounts_control_fp = (
    resource_dir / "parnet_encore_eclip" / "bigwig_processed_counts" / "{RBP_CL}" / "control" / "counts.{STRAND}.bw"
)


In [ ]:
globbed_wildcards = glob_wildcards(ufmt_readcounts_eclip_fp)

# Here we use a function that automatically identifies the possible values for "wildcards" in a filepath.

df_experiments = pd.DataFrame(globbed_wildcards._asdict())
df_experiments = pd.concat(
    [df_experiments, df_experiments["RBP_CL"].str.split("_", expand=True).set_axis(["RBP", "CL"], axis=1)], axis=1
)
df_experiments = (
    df_experiments.sort_values(by=["RBP", "CL", "REPLICATE", "STRAND"], ascending=[True, True]).reset_index(drop=True),
)

display(df_experiments.head(5))

ValueError: Length of ascending (2) != length of by (4)

In [ ]:
for rbp_ct, replicate, strand in zip(*glob_wildcards(ufmt_eclip_fp)):
    print(rbp_ct, replicate, strand)
    break

In [ ]:
for RBP_CT in set(glob_wildcards(ufmt_eclip_fp).RBP_CT):
    if not os.path.exists(str(ufmt_eclip_fp).format(RBP_CT=RBP_CT, REPLICATE="2", STRAND="pos")):
        print(f"eCLIP data for {RBP_CT} not found, skipping.")
        continue

## eCLIP - peaks

## Integrating together - distribution of read counts in peaks

### Read counts distribution in peaks

### Peaks distribution accross gene types

## External resources

### oRNAment

Database that provides pre-computed instances of binding sites for a large number of RBPs. 

This may be useful to compare with your own results. 

e.g. here PTBP1 is shown to rather bind in protein coding genes, specifically in the 3'UTR region.

In [ ]:
display(Image("../documentation/media/ornament.ptbp1_example.png"))

### mCrossBase

[https://zhanglab.c2b2.columbia.edu/mCrossBase/rbp.php?id=K562.RBFOX2](https://zhanglab.c2b2.columbia.edu/mCrossBase/rbp.php?id=K562.RBFOX2)

In [ ]:
display(Image("../documentation/media/mcrossbase.rbfox2_example.png"))

In [ ]:
url = "https://zhanglab.c2b2.columbia.edu/mCrossBase/static/plots/logo/{CELLTYPE}.{RBP}.top10.cluster.m1.00.svg".format(
    RBP="RBFOX2", CELLTYPE="K562"
)
response = requests.get(url, verify=False)

display(SVG(response.content))

## Cleanup

In [ ]:
pbt.cleanup()